In [1]:
import pandas as pd
import numpy as np

In [2]:
# 层次化索引是pandas的一项重要功能，他使你能在一个轴上拥有多个索引级别
# 创建一个Series，并用一个由列表或数组组成的列表作为索引
data = pd.Series(np.random.randn(9),
                 index = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                          [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.400409
   2    0.731772
   3   -0.626240
b  1    0.124789
   3   -1.074833
c  1    1.020496
   2   -0.585532
d  2   -0.866475
   3   -0.400676
dtype: float64

In [3]:
# 看到的结果是经过美化的带有MultiIndex索引的Series的格式。索引之间的“间隔”表示“直接用上面的标签”
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [ ]:
# codes 第一个列表 外层索引   第二个 内层索引

In [4]:
# 对一个层次化索引的对象，可以使用所谓的部分索引
data['b']

1    0.124789
3   -1.074833
dtype: float64

In [5]:
data['b': 'c']

b  1    0.124789
   3   -1.074833
c  1    1.020496
   2   -0.585532
dtype: float64

In [6]:
data[['b', 'd']]

b  1    0.124789
   3   -1.074833
d  2   -0.866475
   3   -0.400676
dtype: float64

In [7]:
# 有时还可以在‘内层’中进行选取
data[:, 2]

a    0.731772
c   -0.585532
d   -0.866475
dtype: float64

In [8]:
# 层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。
# 例如，可以通过unstack方法将这段数据重新安排到一个DataFrame中
data.unstack()

,1,2,3
a,-0.400409,0.731772,-0.626240
b,0.124789,NaN,-1.074833
c,1.020496,-0.585532,NaN
d,NaN,-0.866475,-0.400676


In [9]:
# unstack的逆运算是stack
data.unstack().stack()

a  1   -0.400409
   2    0.731772
   3   -0.626240
b  1    0.124789
   3   -1.074833
c  1    1.020496
   2   -0.585532
d  2   -0.866475
   3   -0.400676
dtype: float64

In [10]:
# 对于一个DataFrame，每条轴都可以有分层索引
frame = pd.DataFrame(np.arange(12).reshape(4, 3),
                     index = [['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns = [['Ohio', 'Ohio', 'Colorado'],
                                ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
# 各层都可以有名字（可以是字符串，也可以是别的Python对象）
# 如果指定了名称，它们就会显示在控制台输出中
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
# 有了部分列索引，可以轻松选取列分组
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [15]:
# 可以单独创建MultiIndex然后复用。 上面那个DataFrame中的列可以这样创建
x = pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                        names = ['states', 'colors'])

In [22]:
frame = pd.DataFrame( np.arange(12).reshape(4, 3),
                      index = [['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                      columns = x )
frame.index.names = ['key1', 'key2']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [27]:
# 重排与分级排序
# 有时你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序
# swaplevel接受两个级别编号或名称，并返回一个互换了级别的新对象（数据不变）
frame.swaplevel('state', 'color', axis = 1)

color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

In [28]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [29]:
# 而sort_index则根据单个级别中的值对数据进行排序
frame.sort_index( level = 1 )  #key2

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [30]:
# 交换级别时，常常也会用到sort_index，这样最终结果就是按照指定顺序进行字母排序了
frame.swaplevel(0, 1).sort_index(level = 0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [31]:
# 根据级别汇总统计
# 许多对DataFrame和Series的描述和汇总统计都有一个level选项，它用于指定某条轴上
# 求和的级别。再以上面那个DataFrame为例，我们可以根据行或列上的级别来进行求和
frame.sum(level = 'key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [34]:
frame.sum(level ='color' , axis = 1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [36]:
#两条轴
frame.sum(level =['color'] , axis = 1).sum(level = 'key1')

color,Green,Red
key1,,
a,10,5
b,34,17


In [38]:
# 人们经常想要将DataFrame的一个或多个列当做行索引来用，或者可能希望将行索引变成
# DataFrame的列。以下以这个DataFrame为例
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [39]:
# DataFrame的set_index函数会将其一个或多个列转换为行索引，并创建一个新的DataFrame
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [40]:
# 默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来
frame.set_index(['c', 'd'], drop = False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [41]:
# reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列里面
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [1]:
import pandas as pd
import numpy as np

In [5]:
# pandas对象中的数据可以通过一些方式进行合并
# 数据集的合并（merge）或连接（join）运算时通过一个或多个键将行链接起来的
# 这些运算是关系型数据库（基于SQL）的核心。
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [3]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [6]:
# 这是一种多对一的合并。df1中的数据有多个被标记为a和b的行，而df2中key列的
# 每个值则仅对应一行。对这些对象调用merge即可得到
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [10]:
# 如果没有指定要用哪个列进行连接，merge就会将重叠列的列名当做键，不过最好明确一下
pd.merge(df1, df2, on = 'key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [16]:
# 如果两个对象的列名不同，也可以分别进行指定
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [19]:
# 可能你已经注意到了，结果里面c和d以及与之相关的数据消失了。默认情况下，merge
# 做的是‘内连接’，结果中的键是交集。其他方式还有‘left’，right’以及‘outer’
# 外连接求取的是键的并集，组合了左连接和右连接的效果
pd.merge(df1, df2, how = 'outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [ ]:
# inner 交集（默认）  left 左表所有键  right  右表所有键  outer两个表所有键

In [20]:
# 多对多的合并有些不直观 看下面的例子
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [21]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [22]:
pd.merge(df1, df2, on = 'key', how = 'left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [23]:
# 多对多连接产生的是行的笛卡尔积。由于左边的DataFrame有3个‘b’行，右边的有2个，
# 所以最终结果就有6个‘b’行，连接方式只影响出现在结果中的不同的键的值
pd.merge(df1, df2, how = 'inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [25]:
# 要根据多个键进行合并，传入一个由列名组成的列表即可
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on = ['key1'], how = 'outer')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [26]:
pd.merge(left, right, on = ['key1', 'key2'], how = 'outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [27]:
# 合并运算需要考虑的最后一个问题是对重复列名的处理。虽然可以手工处理列名重叠的
# 问题（查看前面介绍的重命名轴标签）， 但merge有一个更实用的suffixes选项，用于
# 指定附加到左右两个DataFrame对象的重叠列名上的字符串
pd.merge(left, right, on ='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [28]:
pd.merge(left, right, on = 'key1', suffixes = ('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [2]:
# 有时候DataFrame中的连接键位于其索引中，在这种情况下，你可以传入left_index=True
# 或right_index=True（或两个都传）以说明索引应该被用作连接键
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value':range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index = ['a', 'b'])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [3]:
right1

,group_val
a,3.5
b,7.0


In [6]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [7]:
# 由于默认的merge方法是求取连接键的交集，因此你可以通过外连接的方式得到他们的并集
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [8]:
# 对于层次化索引的数据，事情就有点复杂了，因为索引的合并默认是多键合并
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

In [10]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [11]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [12]:
# 这种情况下，你必须以列表的形式指明用作合并键的多个列
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [13]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [14]:
# 同时使用合并双方的索引也没问题
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [15]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [16]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [17]:
# DataFrame还有一个便捷的join实例方法，它能更为方便地实现按索引合并。它还可用于
# 合并多个带有相同或相似索引的DataFrame对象，但要求没有重叠的列。
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [20]:
# 因为一些历史版本的遗留原因，DataFrame的join方法默认使用的是左连接，保留左边表
# 的行索引，它还支持在调用的DataFrame的列上连接传递的DataFrame索引
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [21]:
# 对于简单的索引合并，你可以向join传入一组DataFrame，同时concat函数也能实现该功能
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [22]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [23]:
left2.join([right2, another], how='outer')

F:\Anaconda\lib\site-packages\pandas\core\frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


In [2]:
# 轴向连接
# 另一种数据合并运算也被称作连接（concatenation）、绑定（binding）或堆叠（stacking）
# Numpy的concatenation函数可以用Numpy数组来做
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [3]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [4]:
np.concatenate([arr, arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [5]:
# 对于pandas对象（如Series和DataFrame），带有标签的轴使你能够进一步推广数组的
# 连接运算。
# pandas的concat函数提供了一种能够解决这些问题的可靠方式，假设有三个没有重叠索引的Series
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
# 对这些对象调用concat可以将值和索引粘合在一起
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [6]:
# 默认情况下，concat是在axis=0上工作的，最终产生一个新的Series。
# 如果传入axis=1，则结果就会变成一个DataFrame
pd.concat([s1, s2, s3], axis=1)

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [7]:
# 这种情况下，另外的轴上没有重叠，从索引的有序并集上就可以看出来。传入join=‘inner’
# 即可得到它们的交集
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [8]:
pd.concat([s1, s4], axis=1)

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [9]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [11]:
# 在这个例子中，f和g标签消失了，是因为使用的是join=‘inner’选项
# 你可以通过join_axes指定要在其他轴上使用的索引
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [12]:
# 不过有个问题，参与连接的片段在结果中区分不开。假设你想要在连接轴上创建一个层次化索引，
# 使用keys参数即可达到这个目的
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [13]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [14]:
# 如果沿着axis=1对Series进行合并，则keys就会成为DataFrame的列头
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [15]:
# 同样的逻辑也适用于DataFrame对象
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [16]:
df2

,three,four
a,5,6
c,7,8


In [17]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [18]:
# 如果传入的不是列表而是一个字典，则字典的键就会被当做keys选项的值
pd.concat({'level1': df1, 'level2': df2}, axis=1)

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [19]:
# 我们可以用names参数命名创建的轴级别
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [20]:
# 最后一个关于DataFrame的问题是，DataFrame的行索引不包含任何相关数据
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [21]:
df1

,a,b,c,d
0,-0.674640,0.151608,-0.679033,-0.327256
1,-1.054484,-0.964388,0.147703,-0.832692
2,-1.321535,0.289071,0.703049,1.231538


In [22]:
df2

,b,d,a
0,0.530372,0.483140,-0.450520
1,-2.314528,0.260487,1.215825


In [23]:
# 在这种情况下，传入ignore_index=True即可
pd.concat([df1, df2], ignore_index=True)

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,a,b,c,d
0,-0.674640,0.151608,-0.679033,-0.327256
1,-1.054484,-0.964388,0.147703,-0.832692
2,-1.321535,0.289071,0.703049,1.231538
3,-0.450520,0.530372,NaN,0.483140
4,1.215825,-2.314528,NaN,0.260487


In [24]:
pd.concat([df1, df2])

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-0.674640,0.151608,-0.679033,-0.327256
1,-1.054484,-0.964388,0.147703,-0.832692
2,-1.321535,0.289071,0.703049,1.231538
0,-0.450520,0.530372,NaN,0.483140
1,1.215825,-2.314528,NaN,0.260487


In [25]:
# 合并重叠数据
# 还有一种数据组合问题不能用简单的合并（merge）或连接（concatenation）运算来处理
# 比如说，你可能有索引全部或部分重叠的两个数据集，我们使用numpy的where函数
# 他表示一种等价于面向数组的if-else
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1]=np.nan
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [26]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [27]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [28]:
# Series有一个combine_first方法，实现的也是一样的功能，还带有pandas的数据对齐
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [29]:
# 对于DataFrame，combine_first自然也会在列上做同样的事情，因此你可以将其看做：
# 用传递对象中的数据为调用对象的缺失数据‘打补丁’
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [30]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [31]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN
